# Load Libraries


In [3]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()

True

# Create snowflake connection with sqlalchemy

In [4]:
snowflake_account = os.getenv('snowflake_account')
snowflake_user = os.getenv('snowflake_user')
snowflake_password = os.getenv('snowflake_password')
snowflake_warehouse = os.getenv('snowflake_warehouse')
snowflake_schema = os.getenv('snowflake_schema')

engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=snowflake_user,
        password=snowflake_password,
        account_identifier=snowflake_account,
    )
)
connection = engine.connect()


# Create Database

In [5]:
target_table = 'TopicDetails'

target_database = 'CFAInstitute'

create_database_query = f"CREATE DATABASE IF NOT EXISTS {target_database}"

connection.execute(create_database_query)
use_database_query = f"USE DATABASE {target_database}"
connection.execute(use_database_query)
connection.execute("USE WAREHOUSE TEST")


/var/folders/6q/q891flcj0r375hpjwrt2wtbm0000gn/T/ipykernel_78179/203564349.py:7: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  connection.execute(create_database_query)


# Create Table

In [6]:
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {target_table} (
    Title VARCHAR,
    Curriculum VARCHAR,
    Level VARCHAR,
    Topics VARCHAR,
    "Learning Outcomes Section" VARCHAR,
    Introduction VARCHAR,
    "Summary Bullets" VARIANT,
    pdf_link VARCHAR
)
"""

connection.execute(create_table_query)


# stage file

In [7]:
connection.execute(f"TRUNCATE TABLE {target_table}")

connection.execute(f"PUT file://resources/processed_content.csv @%{target_table}")


# Load data from stage

In [8]:
connection.execute(f"COPY INTO {target_table} ON_ERROR=CONTINUE FILE_FORMAT = (FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER=1 PARSE_HEADER = FALSE)")